In [1]:
import cv2 as cv
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
%matplotlib inline
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
import pandas as pd

In [3]:
def plot_image(img, cmap='gray'):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img, cmap=cmap)
    plt.show()

In [4]:
df = pd.read_pickle('data/df.pkl')
final_df = pd.read_csv('data/final_df.csv')

In [5]:
from torchvision.models import resnet18
from torchvision.datasets import EMNIST
from torchvision import transforms
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [7]:
model = Net()
model.load_state_dict(torch.load('data/model.pth'))
model.eval()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [8]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'J', 'K', 'Q']
classes_map = {'0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'J':10, 'K':11, 'Q':12}
pred_dict = {c1:{c2:0 for c2 in classes} for c1 in classes}

In [9]:
from tqdm import tqdm

X = []
y = []

with torch.no_grad():
    for i in tqdm(range(91)):

        data_row = df.iloc[i]
        truth_row = final_df.iloc[i]
        cards = ['P1_number', 'P2_number', 'P3_number', 'P4_number']
        
        for idx in cards:
            vector = cv.resize(data_row[idx], (28, 28))
            card = torch.tensor(vector, dtype=torch.float).unsqueeze(0).unsqueeze(0)
            truth = truth_row[idx]    
            out = model(card)
            X.append(out.numpy().reshape(-1))
            y.append(classes_map[truth])
            _, predicted_tensor = torch.max(out.data, 1)
            predicted = predicted_tensor.item()
            
            pred_dict[str(predicted)][truth] += 1

100%|██████████| 91/91 [00:00<00:00, 406.70it/s]


In [10]:
df_pred = pd.DataFrame(pred_dict)

In [11]:
df_pred

,0,1,2,3,4,5,6,7,8,9,J,K,Q
0,24,2,2,0,0,0,0,0,0,0,0,0,0
1,0,24,0,0,0,0,0,3,1,0,0,0,0
2,2,0,25,0,0,0,0,1,0,0,0,0,0
3,0,0,0,27,0,0,0,1,1,0,0,0,0
4,0,0,0,0,26,0,1,1,0,0,0,0,0
5,0,0,1,2,0,25,0,0,0,0,0,0,0
6,1,3,0,0,0,0,27,0,1,0,0,0,0
7,0,1,3,0,0,0,0,22,0,0,0,0,0
8,0,0,0,1,0,0,0,1,26,0,0,0,0
9,0,0,1,1,0,0,0,5,6,11,0,0,0


In [16]:
precision_vector = df_pred.apply(collapse_axis, axis=0)
recall_vector = df_pred.apply(collapse_axis, axis=1)

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


In [17]:
print("Precision: ", precision_vector)
print("Recall: ", recall_vector)

Precision:  0    0.888889
1    0.571429
2    0.714286
3    0.870968
4    0.742857
5    1.000000
6    0.375000
7    0.611111
8    0.520000
9    1.000000
J         NaN
K         NaN
Q         NaN
dtype: float64
Recall:  0    0.857143
1    0.857143
2    0.892857
3    0.931034
4    0.928571
5    0.892857
6    0.843750
7    0.846154
8    0.928571
9    0.458333
J    0.000000
K    0.000000
Q    0.000000
dtype: float64


In [18]:
def collapse_axis(row):
    return row[row.name] / row.sum()
    

In [19]:
X = np.array(X)
y = np.array(y)

In [20]:
pred_dict = {i:{j:0 for j in range(13)} for i in range(13)}

In [21]:
import xgboost
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import cross_val_score

In [22]:
model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:10] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:10] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:10] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:10] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:10] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 85.72% (3.28%)


In [23]:
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
for train_index, test_index in kfold.split(range(X.shape[0])):
    model = xgboost.XGBClassifier()
    X_train = X[train_index, :]
    y_train = y[train_index]
    X_test = X[test_index, :]
    y_test = y[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    for i in range(len(y_pred)):
         pred_dict[y_pred[i]][y_test[i]] += 1

[10:36:11] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:11] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:11] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:11] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:12] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:106

In [24]:
df_pred = pd.DataFrame(pred_dict)

In [25]:
precision_vector = df_pred.apply(collapse_axis, axis=0).values
recall_vector = df_pred.apply(collapse_axis, axis=1).values

In [26]:
print("Precision: ", precision_vector)
print("Recall: ", recall_vector)

Precision:  [0.92307692 0.75862069 0.82142857 0.93103448 0.92592593 0.92857143
 0.96428571 0.8        0.83333333 0.61538462 0.8125     0.9
 0.92307692]
Recall:  [0.85714286 0.78571429 0.82142857 0.93103448 0.89285714 0.92857143
 0.84375    0.76923077 0.89285714 0.66666667 0.92857143 0.93103448
 0.85714286]


In [27]:
2 * (precision_vector * recall_vector) / (precision_vector + recall_vector)

array([0.88888889, 0.77192982, 0.82142857, 0.93103448, 0.90909091,
       0.92857143, 0.9       , 0.78431373, 0.86206897, 0.64      ,
       0.86666667, 0.91525424, 0.88888889])

In [28]:
df_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,24,2,2,0,0,0,0,0,0,0,0,0,0
1,0,22,0,0,0,0,0,0,1,4,1,0,0
2,1,0,23,0,0,1,0,2,0,0,1,0,0
3,0,0,1,27,0,0,0,0,0,1,0,0,0
4,0,1,0,0,25,0,1,0,0,1,0,0,0
5,0,0,0,2,0,26,0,0,0,0,0,0,0
6,1,2,0,0,0,0,27,0,1,0,0,0,1
7,0,1,2,0,0,0,0,20,0,3,0,0,0
8,0,1,0,0,0,0,0,0,25,0,2,0,0
9,0,0,0,0,1,1,0,3,2,16,1,0,0


In [29]:
from skimage.morphology import erosion, dilation, opening, closing
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import remove_small_objects 
from skimage.morphology import (disk, square, diamond)

In [30]:
def calculate_area(chain):
    area = 0
    x = 0
    y = 0
    x_coord = [None]*len(chain)
    y_coord = [None]*len(chain)
    
    for i in range(len(chain)): # Transforming the chain code into x and y coordinates to compute the area
        if chain[i] == 0:
            x -= 1
            y += 1
        elif chain[i] == 1:
            y += 1
        elif chain[i] == 2:
            x += 1
            y += 1
        elif chain[i] == 3:
            x += 1
        elif chain[i] == 4:
            x += 1
            y -= 1
        elif chain[i] == 5:
            y -= 1
        elif chain[i] == 6:
            x -= 1
            y -= 1
        elif chain[i] == 7:
            x -= 1
        else:
            x = 1000 # Big value to clearly indicate if an error occurs
            y = 1000
        x_coord[i] = x
        y_coord[i] = y
        
    a = 0    
    for j in range(len(chain)-1): # Calculation of area contributed by consecutive pixels.
        a += x_coord[j]*y_coord[j+1] - x_coord[j+1]*y_coord[j]
    a += x_coord[len(chain)-1]*y_coord[0] - x_coord[0]*y_coord[len(chain)-1]
    
    area = abs(a)/2
    
    return area

In [31]:
def get_chain_code(image):
    feat_vect1 = []
    feat_vect2 = []
    feat_vect3 = []
    feat_vect4 = []
    chain_area = 0
    # Erosion to get rid of small smudges off to the sides.
    erosion_number = erosion(image, selem=square(1), out=None)>0
    # Clean the images by removing image regions that are smaller than 30 pixels
    cleaned_number = remove_small_objects(erosion_number, min_size=30, connectivity=1, in_place=False)
    # Dilate the image around a square with side 1
    dilation_number = dilation(cleaned_number, selem=square(1), out=None)

    img = 255*dilation_number # Dilation step returns True/False values: have to change it back to 0 to 255
    
    start_point = (0,0)
    ## Discover the first point 
    for i, row in enumerate(img):
        for j, value in enumerate(row):
            if value == 255:
                start_point = (i, j)
                break
        else:
            continue
        break

    directions = [ 0,  1,  2,
                   7,      3,
                   6,  5,  4]
    dir2idx = dict(zip(directions, range(len(directions))))

    change_j =   [-1,  0,  1, # x or columns
                  -1,      1,
                  -1,  0,  1]

    change_i =   [-1, -1, -1, # y or rows
                   0,      0,
                   1,  1,  1]

    border = []
    chain = []
    curr_point = start_point
    for direction in directions:
        idx = dir2idx[direction]
        new_point = (start_point[0]+change_i[idx], start_point[1]+change_j[idx])
        if img[new_point] != 0:
            border.append(new_point)
            chain.append(direction)
            curr_point = new_point
            break

    count = 0
    while curr_point != start_point:
        #figure direction to start search
        b_direction = (direction + 5) % 8 
        dirs_1 = range(b_direction, 8)
        dirs_2 = range(0, b_direction)
        dirs = []
        dirs.extend(dirs_1)
        dirs.extend(dirs_2)
        for direction in dirs:
            idx = dir2idx[direction]
            new_point = (curr_point[0]+change_i[idx], curr_point[1]+change_j[idx])
            if img[new_point] != 0: # if is ROI
                border.append(new_point)
                chain.append(direction)
                curr_point = new_point
                break
        if count == 15000: break
        count += 1

    chain_area = calculate_area(chain)
    
    d0=0
    d1=0
    d2=0
    d3=0
    d4=0
    d5=0
    d6=0
    d7=0
    
    for m in range(len(chain)): # Determines quantity of each direction
        if chain[m] == 0:
            d0 += 1
        elif chain[m] == 1:
            d1 += 1
        elif chain[m] == 2:
            d2 += 1
        elif chain[m] == 3:
            d3 += 1
        elif chain[m] == 4:
            d4 += 1
        elif chain[m] == 5:
            d5 += 1
        elif chain[m] == 6:
            d6 += 1
        elif chain[m] == 7:
            d7 += 1
        else:
            break
            
    directions_count = [d0,d1,d2,d3,d4,d5,d6,d7]
    
    # The deviation gives a quantity to how much the contours makes turns, i.e. a contour with a lot of curves
    # will have a big number, and contours like the number 1, will have smaller numbers.
    deviation = 0
    for i in range(len(chain)-1):
        deviation = deviation + abs(chain[i]-chain[i+1])
      
    return count, border, directions_count, chain_area, deviation

In [39]:
X_chain = []
y_chain = []
for i in range(91):
    
    data_row = df.iloc[i]
    truth_row = final_df.iloc[i]
    cards = ['P1_number', 'P2_number', 'P3_number', 'P4_number']
    
    for index in cards:
        img = data_row[index]
        img = np.pad(img, ((1, 1), (1, 1)), 'constant', constant_values=((0, 0), (0, 0)))
        truth = truth_row[index]
        _, _, directions_count, area, indiv_deviation = get_chain_code(img)
        code_features = directions_count + [area] + [indiv_deviation]
        X_chain.append(code_features)
        y_chain.append(classes_map[truth])
            
        
X_chain = np.array(X_chain)
y_chain = np.array(y_chain)

In [40]:
final_df.head()

,P1_suite,P1_number,P2_suite,P2_number,P3_suite,P3_number,P4_suite,P4_number,D,game
0,S,Q,H,8,H,J,S,0,1,game1
1,S,5,S,J,C,9,D,K,1,game1
2,D,1,H,3,C,7,D,3,1,game1
3,S,1,D,J,S,4,C,4,1,game1
4,D,0,S,6,C,3,C,2,1,game1


In [41]:
model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, X_chain, y_chain, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:35] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:35] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:35] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:35] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:36] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 71.15% (4.95%)


In [42]:
X_both = np.concatenate((X, X_chain), axis=1)

In [43]:
X_both.shape

(364, 20)

In [44]:
model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, X_both, y_chain, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

[10:36:36] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:36] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:36] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:37] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:36:37] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 89.84% (4.12%)


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [45]:
from sklearn.decomposition import PCA

In [46]:
cards = ['P1_number', 'P2_number', 'P3_number', 'P4_number']
cards_df = df[cards]

In [47]:
X_chain = []
y_chain = []
for i in range(91):
    
    data_row = df.iloc[i]
    truth_row = final_df.iloc[i]
    cards = ['P1_number', 'P2_number', 'P3_number', 'P4_number']
    
    for index in cards:
        img = data_row[index]
        img = np.pad(img, ((1, 1), (1, 1)), 'constant', constant_values=((0, 0), (0, 0)))
        truth = truth_row[index]
        _, _, directions_count, area, indiv_deviation = get_chain_code(img)
        code_features = directions_count + [area] + [indiv_deviation]
        X_chain.append(code_features)
        y_chain.append(classes_map[truth])
            
        
X_chain = np.array(X_chain)
y_chain = np.array(y_chain)

In [49]:
pca_ready = []
y_pca = []

for i in tqdm(range(91)):

    data_row = df.iloc[i]
    truth_row = final_df.iloc[i]
    cards = ['P1_number', 'P2_number', 'P3_number', 'P4_number']

    for idx in cards:
        vector = cv.resize(data_row[idx], (28, 28)).reshape(-1)
        truth = truth_row[idx]    
        pca_ready.append(vector)
        y_pca.append(classes_map[truth])
        
pca_ready = np.array(pca_ready)
y_pca = np.array(y_pca)

100%|██████████| 91/91 [00:00<00:00, 3364.79it/s]


In [50]:
pca_ready.shape

(364, 784)

In [51]:
pca = PCA(n_components=10)
X_pca = pca.fit_transform(pca_ready)

In [52]:
X_pca.shape

(364, 10)

In [56]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, X_pca, y_pca, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:38:54] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:38:54] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:38:54] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:38:54] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:38:54] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 74.99% (8.28%)


In [57]:
X_all = np.concatenate((X, X_pca, X_chain), axis=1)

In [58]:
model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, X_all, y_pca, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

[10:38:55] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-

[10:38:55] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:38:55] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:38:55] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:38:55] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 88.73% (3.54%)


In [90]:
# Fourier Descriptor
def fourierDescriptor(contour_array):
    contour_complex = np.empty(contour_array.shape[0],dtype=complex)
    contour_complex.real = contour_array[:,0,0]
    contour_complex.imag = contour_array[:,0,1]
    fourier_result= np.fft.fft(contour_complex)
    magnitude = np.array([np.abs(fft) for fft in fourier_result][1:11])
    r = magnitude[0]
    magnitude = [m / r for m in magnitude]
    return magnitude

In [91]:
X_fft = []
y_fft = []
for i in range(91):
    
    data_row = df.iloc[i]
    truth_row = final_df.iloc[i]
    cards = ['P1_number', 'P2_number', 'P3_number', 'P4_number']
    
    for index in cards:
        img = data_row[index]
        contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        contours = sorted(contours, key=lambda contour: cv.contourArea(contour), reverse= True)[0]
        truth = truth_row[index]
        fft_features = fourierDescriptor(contours)
        X_fft.append(fft_features)
        y_fft.append(classes_map[truth])
            
        
X_fft = np.array(X_fft)
y_fft = np.array(y_fft)

In [92]:
model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
results = cross_val_score(model, X_fft, y_pca, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:53:21] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:53:21] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:53:22] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:53:22] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:53:22] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 82.97% (6.95%)


In [99]:
X_fft.shape

(364, 10)

In [98]:
X_pca.shape

(364, 10)

In [96]:
from sklearn.preprocessing import MinMaxScaler

In [117]:
X_final = np.concatenate((X_fft, X), axis=1)
model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=179)
results = cross_val_score(model, X_final, y_pca, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:25:59] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:00] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:26:00] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:26:00] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:26:00] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 94.78% (2.65%)
